In [1]:
import numpy as np
import pandas as pd
from matplotlib import pyplot as plt
from sklearn.svm import OneClassSVM
from sklearn.metrics import confusion_matrix
from sklearn.ensemble import IsolationForest
from keras.layers import Input, Dense
from keras.models import Model
from sklearn.model_selection import train_test_split
from numpy.random import seed

/anaconda/envs/py35/lib/python3.5/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [2]:
def acc_f1(y_true, y_pred, abnormal = 0):
    cm = np.zeros((2,2))
    cm[0][0] = np.sum((y_true != abnormal)&(y_pred != abnormal))
    cm[0][1] = np.sum((y_true != abnormal)&(y_pred == abnormal))
    cm[1][0] = np.sum((y_true == abnormal)&(y_pred != abnormal))
    cm[1][1] = np.sum((y_true == abnormal)&(y_pred == abnormal))
    precision = cm[1][1] / np.sum(cm[:,1])
    recall = cm[1][1] / np.sum(cm[1,:])
    f1 = 2*precision*recall/(precision+recall)
    return (cm[0][0] + cm[1][1])/(np.sum(cm)), f1
def IOU(y_true, y_pred, abnormal = 0):
    U = np.sum((y_true == abnormal)|(y_pred == abnormal))
    I = np.sum((y_true == abnormal)&(y_pred == abnormal))
    return I/U

In [3]:
data1_size = pd.read_csv('./../../problem1/train01.csv')
data1_size = data1_size.iloc[:,1:]
data2_size = pd.read_csv('./../../problem1/train02.csv')
data2_size = data2_size.iloc[:,1:]
data3_size = pd.read_csv('./../../problem1/train03.csv')
data3_size = data3_size.iloc[:,1:]
data4_size = pd.read_csv('./../../problem1/train04.csv')
data4_size = data4_size.iloc[:,1:]
data5_size = pd.read_csv('./../../problem1/train05.csv')
data5_size = data5_size.iloc[:,1:]
data6_size = pd.read_csv('./../../problem1/train06.csv')
data6_size = data6_size.iloc[:,1:]
data7_size = pd.read_csv('./../../problem1/train07.csv')
data7_size = data7_size.iloc[:,1:]
data8_size = pd.read_csv('./../../problem1/train08.csv')
data8_size = data8_size.iloc[:,1:]
data9_size = pd.read_csv('./../../problem1/train09.csv')
data9_size = data9_size.iloc[:,1:]
data10_size = pd.read_csv('./../../problem1/train10.csv')
data10_size = data10_size.iloc[:,1:]
data11_size = pd.read_csv('./../../problem1/train11.csv')
data11_size = data11_size.iloc[:,1:]
data12_size = pd.read_csv('./../../problem1/train12.csv')
data12_size = data12_size.iloc[:,1:]
data13_size = pd.read_csv('train13.csv')
data13_size = data13_size.iloc[:,1:]
data14_size = pd.read_csv('train14.csv')
data14_size = data14_size.iloc[:,1:]
data15_size = pd.read_csv('train15.csv')
data15_size = data15_size.iloc[:,1:]
data16_size = pd.read_csv('train16.csv')
data16_size = data16_size.iloc[:,1:]
data17_size = pd.read_csv('train17.csv')
data17_size = data17_size.iloc[:,1:]
data18_size = pd.read_csv('train18.csv')
data18_size = data18_size.iloc[:,1:]
data19_size = pd.read_csv('train19.csv')
data19_size = data19_size.iloc[:,1:]
data20_size = pd.read_csv('train20.csv')
data20_size = data20_size.iloc[:,1:]
data21_size = pd.read_csv('train21.csv')
data21_size = data21_size.iloc[:,1:]
data22_size = pd.read_csv('train22.csv')
data22_size = data22_size.iloc[:,1:]
data23_size = pd.read_csv('train23.csv')
data23_size = data23_size.iloc[:,1:]

In [26]:
train=np.concatenate([data1_size,data2_size,data3_size,data4_size,data5_size,data6_size,data7_size,data8_size,data9_size,data10_size,
                     data11_size, data12_size,data13_size,data14_size,data15_size,data16_size,data17_size,data18_size, data19_size,
                     data20_size,data21_size,data22_size, data23_size],axis=0)

In [27]:
train=pd.DataFrame(train)

In [13]:
test1_size = pd.read_csv('./../../problem1/test01.csv')
test1_size = test1_size.iloc[:,1:]
test2_size = pd.read_csv('./../../problem1/test02.csv')
test2_size = test2_size.iloc[:,1:]
test3_size = pd.read_csv('./../../problem1/test03.csv')
test3_size = test3_size.iloc[:,1:]

In [14]:
abnor1_size = pd.read_csv('./../../problem1/test01_PCR.csv')
abnor1_size = abnor1_size.iloc[:,1:]
abnor2_size = pd.read_csv('./../../problem1/test02_smkLim.csv')
abnor2_size = abnor2_size.iloc[:,1:]

In [28]:
test_a=np.concatenate([test1_size,test2_size,test3_size],axis=0)
test_a.shape

(176071, 111)

In [30]:
test_a=pd.DataFrame(test_a)

In [29]:
ab_a=np.concatenate([abnor1_size,abnor2_size],axis=0)
ab_a.shape

(33374, 111)

In [31]:
ab_a=pd.DataFrame(ab_a)

In [32]:
train.shape, test_a.shape, ab_a.shape

((1043638, 111), (176071, 111), (33374, 111))

In [33]:
def del_f(i):
    del train[i]
    del test_a[i]
    del ab_a[i]

In [34]:
del_f(14) #ASMod_mAirPerCylCor

In [35]:
del_f(20) #CoETS_stTrqLim_mp

In [36]:
del_f(65) #MisfDet_ctMifDrivCyc_00_

In [37]:
def nor(data2):
    data_norm2 = (data2 - data2.min()) / (data2.max() - data2.min())
    data_norm2 = data_norm2.fillna(0)
    X_data2 = []
    for i in range(0, data_norm2.shape[0], strides):
        #if (len(X_data2) % 200==0):
        #    print(len(X_data2))
        temp = data_norm2.iloc[i:i+interval].values
        if temp.shape[0] != interval:
            continue
        X_data2.append(np.expand_dims(temp, axis=0))
    X_data2 = np.concatenate(X_data2, axis=0)
    print(X_data2.shape)
    return X_data2

In [38]:
interval = 10
strides = 2

In [39]:
train_a=nor(train)
test_b=nor(test_a)
ab_b=nor(ab_a)

(521815, 10, 108)
(88031, 10, 108)
(16683, 10, 108)


In [40]:
train_n= (np.reshape(train_a, [train_a.shape[0], -1]))
test_n= (np.reshape(test_b, [test_b.shape[0], -1]))
ab_n= (np.reshape(ab_b, [ab_b.shape[0], -1]))

In [41]:
train_n.shape, test_n.shape, ab_n.shape

((521815, 1080), (88031, 1080), (16683, 1080))

In [42]:
### Define the encoder dimension
encoding_dim = 32

In [43]:
# define the number of features
ncol = train_n.shape[1]

In [66]:
input_dim = Input(shape = (ncol, ))

# Encoder Layers
encoded1 = Dense(512, activation = 'relu')(input_dim)
encoded2 = Dense(128, activation = 'relu')(encoded1)
encoded3 = Dense(encoding_dim, activation = 'relu')(encoded2)
    
decoded1 = Dense(128, activation = 'relu')(encoded3)
decoded2 = Dense(512, activation = 'relu')(decoded1)
decoded3 = Dense(ncol, activation = 'sigmoid')(decoded2)
# Combine Encoder and Deocder layers
autoencoder = Model(inputs = input_dim, outputs = decoded3)
# Compile the Model
autoencoder.compile(optimizer = 'adam', loss = 'mse')

In [67]:
autoencoder.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_3 (InputLayer)         (None, 1080)              0         
_________________________________________________________________
dense_7 (Dense)              (None, 512)               553472    
_________________________________________________________________
dense_8 (Dense)              (None, 128)               65664     
_________________________________________________________________
dense_9 (Dense)              (None, 32)                4128      
_________________________________________________________________
dense_10 (Dense)             (None, 128)               4224      
_________________________________________________________________
dense_11 (Dense)             (None, 512)               66048     
_________________________________________________________________
dense_12 (Dense)             (None, 1080)              554040    
Total para

In [69]:
X_train, X_test = train_test_split(train_n, train_size = 0.9, random_state = seed(333))

/anaconda/envs/py35/lib/python3.5/site-packages/sklearn/model_selection/_split.py:2069: FutureWarning: From version 0.21, test_size will always complement train_size unless both are specified.
  FutureWarning)


In [70]:
X_train.shape, X_test.shape

((469633, 1080), (52182, 1080))

In [71]:
autoencoder.fit(X_train, X_train, nb_epoch = 50, batch_size =1000, shuffle = False, validation_data = (X_test, X_test))

/anaconda/envs/py35/lib/python3.5/site-packages/ipykernel/__main__.py:1: UserWarning: The `nb_epoch` argument in `fit` has been renamed `epochs`.
  if __name__ == '__main__':


Train on 469633 samples, validate on 52182 samples
Epoch 1/50
469633/469633 [==============================] - 10s 22us/step - loss: 0.0067 - val_loss: 0.0012
Epoch 2/50
469633/469633 [==============================] - 10s 21us/step - loss: 7.6078e-04 - val_loss: 5.1403e-04
Epoch 3/50
469633/469633 [==============================] - 10s 21us/step - loss: 4.9076e-04 - val_loss: 4.3547e-04
Epoch 4/50
469633/469633 [==============================] - 10s 21us/step - loss: 4.1720e-04 - val_loss: 3.7065e-04
Epoch 5/50
469633/469633 [==============================] - 10s 21us/step - loss: 3.5568e-04 - val_loss: 3.2116e-04
Epoch 6/50
469633/469633 [==============================] - 10s 21us/step - loss: 3.2296e-04 - val_loss: 3.0720e-04
Epoch 7/50
469633/469633 [==============================] - 10s 21us/step - loss: 3.0084e-04 - val_loss: 2.8076e-04
Epoch 8/50
469633/469633 [==============================] - 10s 21us/step - loss: 2.7437e-04 - val_loss: 2.5093e-04
Epoch 9/50
469633/469633 [===

In [72]:
encoder = Model(inputs = input_dim, outputs = encoded3)
encoded_input = Input(shape = (encoding_dim, ))

In [73]:
encoded_train = pd.DataFrame(encoder.predict(train_n))
encoded_train = encoded_train.add_prefix('feature_')

encoded_test = pd.DataFrame(encoder.predict(test_n))
encoded_test = encoded_test.add_prefix('feature_')

encoded_ab = pd.DataFrame(encoder.predict(ab_n))
encoded_ab = encoded_ab.add_prefix('feature_')

# one class svm

In [84]:
from sklearn.preprocessing import minmax_scale


In [85]:
train_scaled = minmax_scale(encoded_train, axis = 0)
test_scaled = minmax_scale(encoded_test, axis = 0)
ab_scaled = minmax_scale(encoded_ab, axis = 0)

/anaconda/envs/py35/lib/python3.5/site-packages/ipykernel/__main__.py:1: DataConversionWarning: Data with input dtype float32 were all converted to float64.
  if __name__ == '__main__':
/anaconda/envs/py35/lib/python3.5/site-packages/ipykernel/__main__.py:2: DataConversionWarning: Data with input dtype float32 were all converted to float64.
  from ipykernel import kernelapp as app
/anaconda/envs/py35/lib/python3.5/site-packages/ipykernel/__main__.py:3: DataConversionWarning: Data with input dtype float32 were all converted to float64.
  app.launch_new_instance()


In [86]:
    p_svm = OneClassSVM(max_iter=1000, verbose=True, nu=0.02, kernel = 'rbf', gamma=0.0001)
    p_svm.fit(train_scaled)
    p_svm_test = p_svm.predict(test_scaled)
    set(p_svm_test)
    p_svm_abnor = p_svm.predict(ab_scaled)
    set(p_svm_abnor)
    p_pred = np.concatenate([p_svm_test, p_svm_abnor], axis=0)
    p_Y = np.array([1] * test_scaled.shape[0] + [-1] * ab_scaled.shape[0])
    p_result = acc_f1(p_Y, p_pred, abnormal = -1)
    acc_f1(p_Y, p_pred, abnormal = -1)
    result_IOU = IOU(p_Y, p_pred, abnormal = -1)    
    print("Accuracy: %.8f"%((p_result[1]+p_result[0])/2))
    print("IOU: %.8f"%(result_IOU))

[LibSVM]

/anaconda/envs/py35/lib/python3.5/site-packages/sklearn/svm/base.py:244: ConvergenceWarning: Solver terminated early (max_iter=1000).  Consider pre-processing your data with StandardScaler or MinMaxScaler.
  % self.max_iter, ConvergenceWarning)


Accuracy: 0.21708498
IOU: 0.15931967


In [87]:
acc_f1(p_Y, p_pred, abnormal = -1), IOU(p_Y, p_pred, abnormal = -1)

((0.1593196707221575, 0.27485028460340866), 0.1593196707221575)

In [60]:
    p_svm = OneClassSVM(max_iter=1000, verbose=True, nu=0.02, kernel = 'rbf', gamma=0.001)
    p_svm.fit(encoded_train)
    p_svm_test = p_svm.predict(encoded_test)
    set(p_svm_test)
    p_svm_abnor = p_svm.predict(encoded_ab)
    set(p_svm_abnor)
    p_pred = np.concatenate([p_svm_test, p_svm_abnor], axis=0)
    p_Y = np.array([1] * encoded_test.shape[0] + [-1] * encoded_ab.shape[0])
    p_result = acc_f1(p_Y, p_pred, abnormal = -1)
    acc_f1(p_Y, p_pred, abnormal = -1)
    result_IOU = IOU(p_Y, p_pred, abnormal = -1)    
    print("Accuracy: %.8f"%((p_result[1]+p_result[0])/2))
    print("IOU: %.8f"%(result_IOU))

[LibSVM]

/anaconda/envs/py35/lib/python3.5/site-packages/sklearn/svm/base.py:244: ConvergenceWarning: Solver terminated early (max_iter=1000).  Consider pre-processing your data with StandardScaler or MinMaxScaler.
  % self.max_iter, ConvergenceWarning)


Accuracy: 0.21708498
IOU: 0.15931967


In [61]:
    p_svm = OneClassSVM(max_iter=1000, verbose=True, nu=0.03, kernel = 'rbf', gamma=1)
    p_svm.fit(encoded_train)
    p_svm_test = p_svm.predict(encoded_test)
    set(p_svm_test)
    p_svm_abnor = p_svm.predict(encoded_ab)
    set(p_svm_abnor)
    p_pred = np.concatenate([p_svm_test, p_svm_abnor], axis=0)
    p_Y = np.array([1] * encoded_test.shape[0] + [-1] * encoded_ab.shape[0])
    p_result = acc_f1(p_Y, p_pred, abnormal = -1)
    acc_f1(p_Y, p_pred, abnormal = -1)
    result_IOU = IOU(p_Y, p_pred, abnormal = -1)    
    print("Accuracy: %.8f"%((p_result[1]+p_result[0])/2))
    print("IOU: %.8f"%(result_IOU))

[LibSVM]

/anaconda/envs/py35/lib/python3.5/site-packages/sklearn/svm/base.py:244: ConvergenceWarning: Solver terminated early (max_iter=1000).  Consider pre-processing your data with StandardScaler or MinMaxScaler.
  % self.max_iter, ConvergenceWarning)


Accuracy: 0.21708498
IOU: 0.15931967


# isolation forest

In [91]:
clf3=IsolationForest(contamination=0.5, random_state=333, n_jobs=-1, behaviour= "new", max_features=16, max_samples=100)
clf3.fit(encoded_train)
pred_test3 = clf3.predict(encoded_test)
pred_abnor3= clf3.predict(encoded_ab)

pred_if3 = np.concatenate([pred_test3, pred_abnor3], axis=0)
Y_data_if3 = np.array([1] * encoded_test.shape[0] + [-1] * encoded_ab.shape[0])
result_if3 = acc_f1(Y_data_if3, pred_if3, abnormal = -1)
#acc_f1(Y_data_if3, pred_if3, abnormal = -1)
result_IOU = IOU(Y_data_if3, pred_if3, abnormal = -1)

print("Accuracy: %.8f"%((result_if3[0]+result_if3[1])/2))
print("IOU: %.8f"%(result_IOU))

Accuracy: 0.21708498
IOU: 0.15931967


# Grid Search

In [62]:
from sklearn.model_selection import GridSearchCV
scores = ['precision', 'recall']
tuned_parameters = [{'gamma':[0.00001, 0.0001,0.001, 0.01, 0.1, 1, 10,100],
                    'nu':[0.02, 0.03, 0.035]}]


In [63]:
train_y=np.ones((521815, 1)) ###############수정필

In [ ]:
for score in scores:
    clf_svm = GridSearchCV(OneClassSVM(max_iter=1000, verbose=True, kernel = 'rbf'), tuned_parameters, cv=5, scoring='%s_macro' % score)
    clf_svm.fit(encoded_train, train_y)
   
    means = clf_svm.cv_results_['mean_test_score']
    stds = clf_svm.cv_results_['std_test_score']

[LibSVM]

/anaconda/envs/py35/lib/python3.5/site-packages/sklearn/svm/base.py:244: ConvergenceWarning: Solver terminated early (max_iter=1000).  Consider pre-processing your data with StandardScaler or MinMaxScaler.
  % self.max_iter, ConvergenceWarning)


[LibSVM]

/anaconda/envs/py35/lib/python3.5/site-packages/sklearn/svm/base.py:244: ConvergenceWarning: Solver terminated early (max_iter=1000).  Consider pre-processing your data with StandardScaler or MinMaxScaler.
  % self.max_iter, ConvergenceWarning)


[LibSVM]

...

[LibSVM]

/anaconda/envs/py35/lib/python3.5/site-packages/sklearn/svm/base.py:244: ConvergenceWarning: Solver terminated early (max_iter=1000).  Consider pre-processing your data with StandardScaler or MinMaxScaler.
  % self.max_iter, ConvergenceWarning)


[LibSVM]

/anaconda/envs/py35/lib/python3.5/site-packages/sklearn/svm/base.py:244: ConvergenceWarning: Solver terminated early (max_iter=1000).  Consider pre-processing your data with StandardScaler or MinMaxScaler.
  % self.max_iter, ConvergenceWarning)


[LibSVM]

In [ ]:
for mean, std, params in zip(means, stds, clf_svm.cv_results_['params']):
       print("%0.3f (+/-%0.03f) for %r"% (mean, std * 2, params))
print(clf_svm.best_params_)

# LSTM AUTO ENCODER 

In [110]:
train_a.shape, test_b.shape, ab_b.shape

((521815, 10, 108), (88031, 10, 108), (16683, 10, 108))

In [100]:
from keras.models import Sequential
from keras.layers import LSTM
from keras.layers import RepeatVector
from keras.layers import TimeDistributed

In [95]:
train_n = minmax_scale(encoded_train, axis = 0)
test_n = minmax_scale(encoded_test, axis = 0)
ab_n = minmax_scale(encoded_ab, axis = 0)

/anaconda/envs/py35/lib/python3.5/site-packages/ipykernel/__main__.py:1: DataConversionWarning: Data with input dtype float32 were all converted to float64.
  if __name__ == '__main__':
/anaconda/envs/py35/lib/python3.5/site-packages/ipykernel/__main__.py:2: DataConversionWarning: Data with input dtype float32 were all converted to float64.
  from ipykernel import kernelapp as app
/anaconda/envs/py35/lib/python3.5/site-packages/ipykernel/__main__.py:3: DataConversionWarning: Data with input dtype float32 were all converted to float64.
  app.launch_new_instance()


In [115]:
timesteps =  10
n_features =  train_a.shape[2]
epochs_n = 50
batch_n = 1000
lr = 0.0001
encoding_dim=32

In [119]:
# define model
model = Sequential()
model.add(LSTM(128, activation='relu', input_shape=(timesteps,n_features), return_sequences=True))
model.add(LSTM(64, activation='relu', return_sequences=False))
model.add(RepeatVector(timesteps))
model.add(LSTM(64, activation='relu', return_sequences=True))
model.add(LSTM(128, activation='relu', return_sequences=True))
model.add(TimeDistributed(Dense(n_features)))
model.compile(optimizer='adam', loss='mse')
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_19 (LSTM)               (None, 10, 128)           121344    
_________________________________________________________________
lstm_20 (LSTM)               (None, 64)                49408     
_________________________________________________________________
repeat_vector_5 (RepeatVecto (None, 10, 64)            0         
_________________________________________________________________
lstm_21 (LSTM)               (None, 10, 64)            33024     
_________________________________________________________________
lstm_22 (LSTM)               (None, 10, 128)           98816     
_________________________________________________________________
time_distributed_5 (TimeDist (None, 10, 108)           13932     
Total params: 316,524
Trainable params: 316,524
Non-trainable params: 0
_________________________________________________________________


# lstm encoder 2!!!!!!!!!!!!!!!

In [121]:
n_dimensions=32
timesteps=10
input_dim=train_a.shape[2]

In [123]:
def get_model(x):
    inputs = Input(shape=(timesteps, input_dim))
    encoded = LSTM(n_dimensions, return_sequences=False, name="encoder")(inputs)
    decoded = RepeatVector(timesteps)(encoded)
    decoded = LSTM(input_dim, return_sequences=True, name='decoder')(decoded)

    autoencoder = Model(inputs, decoded)
    encoder = Model(inputs, encoded)
    return autoencoder, encoder

get_model(train_a)


(<keras.engine.training.Model at 0x7f70583c93c8>,
 <keras.engine.training.Model at 0x7f705836b0b8>)

In [125]:
autoencoder, encoder = get_model(n_dimensions)
autoencoder.compile(optimizer='rmsprop', loss='mse', 
                    metrics=['acc', 'cosine_proximity'])

history = autoencoder.fit(train_a, train_a, batch_size=500, epochs=10)


Epoch 1/10
521815/521815 [==============================] - 44s 84us/step - loss: 0.0050 - acc: 0.2065 - cosine_proximity: -0.9883
Epoch 2/10
521815/521815 [==============================] - 41s 79us/step - loss: 0.0012 - acc: 0.2499 - cosine_proximity: -0.9980
Epoch 3/10
521815/521815 [==============================] - 41s 78us/step - loss: 0.0012 - acc: 0.2811 - cosine_proximity: -0.9982
Epoch 4/10
521815/521815 [==============================] - 40s 77us/step - loss: 0.0011 - acc: 0.3115 - cosine_proximity: -0.9983
Epoch 5/10
521815/521815 [==============================] - 40s 77us/step - loss: 0.0011 - acc: 0.3241 - cosine_proximity: -0.9983
Epoch 6/10
521815/521815 [==============================] - 40s 77us/step - loss: 0.0011 - acc: 0.3255 - cosine_proximity: -0.9984
Epoch 7/10
521815/521815 [==============================] - 41s 79us/step - loss: 0.0011 - acc: 0.3228 - cosine_proximity: -0.9984
Epoch 8/10
521815/521815 [==============================] - 41s 78us/step - loss: 0

In [132]:
encoded_train = encoder.predict(train_a)
encoded_test = encoder.predict(test_b)
encoded_ab = encoder.predict(ab_b)

In [133]:
encoded_train.shape, encoded_test.shape, encoded_ab.shape

((521815, 32), (88031, 32), (16683, 32))

In [147]:
clf3=IsolationForest(contamination=0.01, random_state=333, n_jobs=-1, behaviour= "new", max_features=32, max_samples=200)
clf3.fit(encoded_train)
pred_test3 = clf3.predict(encoded_test)
pred_abnor3= clf3.predict(encoded_ab)

pred_if3 = np.concatenate([pred_test3, pred_abnor3], axis=0)
Y_data_if3 = np.array([1] * encoded_test.shape[0] + [-1] * encoded_ab.shape[0])
result_if3 = acc_f1(Y_data_if3, pred_if3, abnormal = -1)
#acc_f1(Y_data_if3, pred_if3, abnormal = -1)
result_IOU = IOU(Y_data_if3, pred_if3, abnormal = -1)

print("Accuracy: %.8f"%((result_if3[0]+result_if3[1])/2))
print("IOU: %.8f"%(result_IOU))

Accuracy: 0.81449375
IOU: 0.55189096


In [151]:
cm = confusion_matrix(Y_data_if3, pred_if3)
cm

array([[10609,  6074],
       [ 2540, 85491]])

In [149]:
train_n = minmax_scale(encoded_train, axis = 0)
test_n = minmax_scale(encoded_test, axis = 0)
ab_n = minmax_scale(encoded_ab, axis = 0)

In [150]:
    p_svm = OneClassSVM(max_iter=500, verbose=True, nu=0.01, kernel = 'rbf', gamma=0.001)
    p_svm.fit(train_n)
    p_svm_test = p_svm.predict(test_n)
    set(p_svm_test)
    p_svm_abnor = p_svm.predict(ab_n)
    set(p_svm_abnor)
    p_pred = np.concatenate([p_svm_test, p_svm_abnor], axis=0)
    p_Y = np.array([1] * test_n.shape[0] + [-1] * ab_n.shape[0])
    p_result = acc_f1(p_Y, p_pred, abnormal = -1)
    acc_f1(p_Y, p_pred, abnormal = -1)
    result_IOU = IOU(p_Y, p_pred, abnormal = -1)    
    print("Accuracy: %.8f"%((p_result[1]+p_result[0])/2))
    print("IOU: %.8f"%(result_IOU))

[LibSVM]

/anaconda/envs/py35/lib/python3.5/site-packages/sklearn/svm/base.py:244: ConvergenceWarning: Solver terminated early (max_iter=500).  Consider pre-processing your data with StandardScaler or MinMaxScaler.
  % self.max_iter, ConvergenceWarning)


Accuracy: 0.21708498
IOU: 0.15931967


# lstm --> grid search isf

In [156]:
from sklearn.model_selection import GridSearchCV
scores = ['precision', 'recall']
tuned_parameters = [{'contamination':[ 0.01, 0.02, 0.03], 
                     'max_samples': [25, 30, 35, 40, 45,100],
                    'random_state':[3, 33, 333],
                    'max_features':[32,16]}]

In [157]:
train_y=np.ones((521815, 1)) ###############수정필

In [ ]:
for score in scores:
    clf = GridSearchCV(IsolationForest(n_jobs=-1, behaviour="new"), tuned_parameters, cv=5, scoring='%s_macro' % score)
    clf.fit(encoded_train, train_y)
   
    means = clf.cv_results_['mean_test_score']
    stds = clf.cv_results_['std_test_score']

/anaconda/envs/py35/lib/python3.5/site-packages/sklearn/metrics/classification.py:1145: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples.
  'recall', 'true', average, warn_for)
/anaconda/envs/py35/lib/python3.5/site-packages/sklearn/metrics/classification.py:1145: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples.
  'recall', 'true', average, warn_for)
/anaconda/envs/py35/lib/python3.5/site-packages/sklearn/metrics/classification.py:1145: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples.
  'recall', 'true', average, warn_for)
/anaconda/envs/py35/lib/python3.5/site-packages/sklearn/metrics/classification.py:1145: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples.
  'recall', 'true', average, warn_for)
/anaconda/envs/py35/lib/python3.5/site-packages/sklearn/metrics/classification.py:1145: Undefine

In [159]:
for mean, std, params in zip(means, stds, clf.cv_results_['params']):
       print("%0.3f (+/-%0.03f) for %r"% (mean, std * 2, params))
print(clf.best_params_)

0.592 (+/-0.409) for {'contamination': 0.01, 'random_state': 3, 'max_samples': 25, 'max_features': 32}
0.493 (+/-0.012) for {'contamination': 0.01, 'random_state': 33, 'max_samples': 25, 'max_features': 32}
0.792 (+/-0.509) for {'contamination': 0.01, 'random_state': 333, 'max_samples': 25, 'max_features': 32}
0.685 (+/-0.516) for {'contamination': 0.01, 'random_state': 3, 'max_samples': 30, 'max_features': 32}
0.488 (+/-0.032) for {'contamination': 0.01, 'random_state': 33, 'max_samples': 30, 'max_features': 32}
0.781 (+/-0.540) for {'contamination': 0.01, 'random_state': 333, 'max_samples': 30, 'max_features': 32}
0.568 (+/-0.444) for {'contamination': 0.01, 'random_state': 3, 'max_samples': 35, 'max_features': 32}
0.484 (+/-0.034) for {'contamination': 0.01, 'random_state': 33, 'max_samples': 35, 'max_features': 32}
0.566 (+/-0.452) for {'contamination': 0.01, 'random_state': 333, 'max_samples': 35, 'max_features': 32}
0.569 (+/-0.439) for {'contamination': 0.01, 'random_state': 3, 

In [196]:
err =encoded_ab.shape[0]/(encoded_train.shape[0]+encoded_test.shape[0])
err

0.027356086618588956

In [209]:
clf3=IsolationForest(contamination=err, random_state=332, n_jobs=-1, behaviour= "new", max_features=16, max_samples=100)
clf3.fit(encoded_train)
pred_test3 = clf3.predict(encoded_test)
pred_abnor3= clf3.predict(encoded_ab)

pred_if3 = np.concatenate([pred_test3, pred_abnor3], axis=0)
Y_data_if3 = np.array([1] * encoded_test.shape[0] + [-1] * encoded_ab.shape[0])
result_if3 = acc_f1(Y_data_if3, pred_if3, abnormal = -1)
#acc_f1(Y_data_if3, pred_if3, abnormal = -1)
result_IOU = IOU(Y_data_if3, pred_if3, abnormal = -1)

print("Accuracy: %.8f"%((result_if3[0]+result_if3[1])/2))
print("IOU: %.8f"%(result_IOU))

Accuracy: 0.85452942
IOU: 0.63627625


In [129]:
# fit model
model.fit(train_a, train_a, epochs=epochs_n, batch_size=batch_n, verbose=0, validation_data = (train_a, train_a) )


InternalError: Dst tensor is not initialized.
	 [[{{node training_3/Adam/gradients/lstm_22/while/mul_1_grad/Mul/StackPopV2}}]]
	 [[{{node loss_5/mul}}]]

In [ ]:
encoded_train = pd.DataFrame(model.predict(train_a))
encoded_train = encoded_train.add_prefix('feature_')

encoded_test = pd.DataFrame(model.predict(test_b))
encoded_test = encoded_test.add_prefix('feature_')

encoded_ab = pd.DataFrame(model.predict(ab_b))
encoded_ab = encoded_ab.add_prefix('feature_')

In [ ]:
# demonstrate reconstruction
yhat = model.predict(X, verbose=0)

In [ ]:
encoder = Model(inputs = n_features, outputs = encoded5)
encoded_input = Input(shape = (encoding_dim, ))